In [398]:
# Libraries
from fpdf import FPDF
from datetime import datetime
import datetime
import pandas as pd
import yaml
import re



In [399]:
# Define the path and filename
yaml_path = "./"
yaml_file = "settings.yaml"

# Construct the full path to the YAML file
full_path = yaml_path + yaml_file

# Load the YAML file
with open(full_path, 'r') as file:
    data = yaml.safe_load(file)

# Access the Title Report
page_title = data.get('report_settings',{}).get('first_page',{}).get('page_title',{})
report_name = data.get('report_settings',{}).get('report_name',{})

In [400]:
class ReportStrategy:
    def __init__(self, header_img_path, chart_img_paths, output_file):
        self.pdf = FPDF()
        self.header_img_path = header_img_path
        # self.tables_path = tables_path
        self.chart_img_paths = chart_img_paths
        self.output_file = output_file

    def add_header(self):
        """Add the header image and title to the PDF"""
        self.pdf.add_page()
        self.pdf.image(self.header_img_path, x=0, y=0, w=210)
        self.pdf.set_text_color(0, 0, 0)  # RGB for black
        self.pdf.set_xy(15, 42)
        self.pdf.add_font('Montserrat-Bold', 'B', './assets/Montserrat/static/Montserrat-Bold.ttf', uni=True)
        self.pdf.set_font('Montserrat-Bold', 'B', 24)
        # self.pdf.multi_cell(0, 10, f"{page_title}", 0, 'C')
        
    def add_table(self):
        """Add CSV table to the report with specific formatting"""
        
        # Load the CSV file into a DataFrame
        csv_file_path = '../data/processed/elasticidades_DLSP_DBGG.csv'  # Replace with your actual file path
        df_header1 = pd.read_csv(csv_file_path).iloc[[1]]
        df_header2 = pd.read_csv(csv_file_path, skiprows=1).iloc[[1]]
        df_numbers = pd.read_csv(csv_file_path, skiprows=1)
        
        # Replace "Unnamed" columns with empty strings in headers
        df_header1.columns = ["" if "Unnamed" in col else col for col in df_header1.columns]
        df_header2.columns = ["" if "Unnamed" in col else col for col in df_header2.columns]
        
        # Set up fonts
        self.pdf.add_font('Montserrat-Bold', 'B', './assets/Montserrat/static/Montserrat-Bold.ttf', uni=True)
        self.pdf.set_font('Montserrat-Bold', 'B', 12)  # Header font
        self.pdf.multi_cell(0, 10, "Elasticidades da DLSP e DBGG", 0, 'L')
        
        # Set up fonts
        self.pdf.add_font('Montserrat-Bold', 'B', './assets/Montserrat/static/Montserrat-Bold.ttf', uni=True)
        self.pdf.set_font('Montserrat-Bold', 'B', 10)  # Header font
        
        # Margins and column widths
        left_margin = 10  # Small left margin
        first_col_width = 50  # Width for the first column
        other_col_width = (self.pdf.w - left_margin * 2 - first_col_width) / (len(df_numbers.columns) - 1)  # Distribute remaining space
        
        # Move to starting position with left margin
        self.pdf.set_x(left_margin)
        
        
        # Write the header row
        for i, column in enumerate(df_header1.columns):
            col_width = first_col_width if i == 0 else other_col_width
            self.pdf.cell(col_width, 8, column, border='B', align="R")
        self.pdf.ln()  # Move to the next row
        
        # Set data font
        self.pdf.add_font('Montserrat-Regular', '', './assets/Montserrat/static/Montserrat-Regular.ttf', uni=True)
        self.pdf.set_font('Montserrat-Regular', '', 9)
        
        # Write the header row
        for i, column in enumerate(df_header2.columns):
            col_width = first_col_width if i == 0 else other_col_width
            column = re.sub(r'\.\d+$', '', column)
            self.pdf.cell(col_width, 10, column, border='B', align="C")
        self.pdf.ln()  # Move to the next row
        
        # Set data font
        self.pdf.add_font('Montserrat-Regular', '', './assets/Montserrat/static/Montserrat-Regular.ttf', uni=True)
        self.pdf.set_font('Montserrat-Regular', '', 9)
        
        for idx, row in df_numbers.iterrows():
            self.pdf.set_x(left_margin)  # Reset position to the left margin
            
            # Check if it's the last row
            is_last_row = idx == df_numbers.index[-1]
            
            for i, item in enumerate(row):
                col_width = first_col_width if i == 0 else other_col_width
                
                # Add a bottom border for the last row
                border_style = 'B' if is_last_row else ''  # 'B' = Bottom border
                
                self.pdf.cell(col_width, 6, str(item), align="C", border=border_style)  # Uniform height for rows
            
            self.pdf.ln()  # Move to the next row
            
        # Add a note below the table in italics
        note_text = "Source: This statistics are produce by the Bacen (Central Bank of Brazil)."
        self.pdf.ln(1)  # Add some space before the note
                # Set the table data using Montserrat-Regular font (or bold if you prefer)
        self.pdf.add_font('Montserrat-Regular', 'I', './assets/Montserrat/static/Montserrat-Italic.ttf', uni=True)
        self.pdf.set_font('Montserrat-Regular', 'I', 8)  # Data font
        self.pdf.set_xy(left_margin, self.pdf.get_y())  # Set the row position
        self.pdf.multi_cell(0, 4, note_text, align="R")  # Write the note text with automatic line breaks

    def add_charts(self):
        """Add the chart images to the PDF"""
        for chart_image_path in self.chart_img_paths:
            y_position = self.pdf.get_y() + 5  # Update the y_position after adding the image
            page_width = self.pdf.w
            image_width = 200
            left_margin = (page_width - image_width) / 2  # Center the image
            self.pdf.image(chart_image_path, x=left_margin, y=y_position, w=image_width)
            # Add a note below the table in italics
            note_text = "Source: Bacen, Author"
            self.pdf.ln(90)  # Add some space before the note
                    # Set the table data using Montserrat-Regular font (or bold if you prefer)
            self.pdf.add_font('Montserrat-Regular', 'I', './assets/Montserrat/static/Montserrat-Italic.ttf', uni=True)
            self.pdf.set_font('Montserrat-Regular', 'I', 8)  # Data font
            self.pdf.set_xy(left_margin, self.pdf.get_y() + 0)  # Set the row position
            self.pdf.multi_cell(0, 0, note_text, align="R")  # Write the note text with automatic line breaks
            

    def generate_report(self):
        """Generate and save the report as a PDF"""
        self.add_header()
        self.add_table()
        self.add_charts()
        self.pdf.output(self.output_file)

In [401]:
# Usage example:
header_img_path = './assets/header.png'
chart_img_paths = [
    '../report/images/primario.png',
    '../report/images/dbgg_chart.png'
]

output_file = f"../report/{report_name}.pdf"

report = ReportStrategy(header_img_path, chart_img_paths, output_file)
report.generate_report()